In [ ]:
import requests
import csv

with open('apikey.txt', 'r') as f:
    API_TOKEN = f.readline()
BASE_URL = "https://api.clashroyale.com/v1"

def get_all_cards():
    url = f"{BASE_URL}/cards"
    headers = {
        "Authorization": f"Bearer {API_TOKEN}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('items', [])
    else:
        return {"error": f"Error {response.status_code}: {response.text}"}

cards = get_all_cards()
card_names = [card['name'] for card in cards]

def card_to_num(card_name):
    for i, name in enumerate(card_names):
        if name == card_name:
            return i
    return -1

def num_to_card(num):
    return card_names[num]


def get_battle_log(session, player_tag):
    sanitized_tag = player_tag.lstrip("#")
    url = f"{BASE_URL}/players/%23{sanitized_tag}/battlelog"
    response = session.get(url)

    if response.status_code == 200:
        return response.json()

    print(f"Error fetching battle log for {player_tag}: {response.status_code} {response.text}")
    return []

def deck_to_nums(deck):
    cards = [card_to_num(card) for card in deck]
    cards.sort()
    return cards

#RANDOM FOREST

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
data = pd.read_csv("matches1M.csv")

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=20, min_samples_leaf=4, min_samples_split=2)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

In [ ]:
def predict_team_odds(team1, team2):
    input_data = np.array(team1 + team2).reshape(1, -1)

    probabilities = model.predict_proba(input_data)

    odds_team1 = probabilities[0][0]
    odds_team2 = probabilities[0][1]

    print(f"Odds of Team 1 winning: {odds_team1 * 100:.2f}%")
    print(f"Odds of Team 2 winning: {odds_team2 * 100:.2f}%")

    return odds_team1, odds_team2

In [ ]:
team1 = [11, 22, 33, 44, 55, 66, 77, 88]
team2 = [99, 88, 77, 66, 55, 44, 33, 22]

#d1 slightly better
#d1 = ["Knight", "Magic Archer", "Goblin Drill", "The Log", "Tornado", "Ice Spirit", "Skeletons", "Cannon"]
#d2 = ["Musketeer", "Skeletons", "The Log", "Fireball", "Ice Spirit", "Cannon", "Hog Rirder", "Ice Golem"]

d1 much better
d1 = ["Goblin Barrel", "Royal Recruits", "Goblinstein", "Cannon Cart", "Goblin Gang", "Dart Goblin", "Arrows", "Cannon"]
d2 = ["Elixir Collector", "Barbarian Hut", "Lightning", "Musketeer", "Zap", "Heal Spirit", "Goblin Machine", "Knight"]
d1 = deck_to_nums(d1)
d2 = deck_to_nums(d2)
predict_team_odds(d1, d2)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [150, 200, 250, 300, 350],
    'max_depth': [15, 20, 25],
    'min_samples_split': [2],
    'min_samples_leaf': [3, 4, 5]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")
model = grid_search.best_estimator_

#XGBOOST GRADIENT BOOSTING







In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, DMatrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [ ]:
data = pd.read_csv("matches.csv")

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

In [ ]:
model = XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
y_pred_proba = model.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)

print(f"Log loss on test set: {loss:.4f}")
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

In [ ]:
def predict_team_odds(team1_players, team2_players):
    if len(team1_players) != 8 or len(team2_players) != 8:
        raise ValueError("Each team must have exactly 8 players.")
    input_data = np.array(team1_players + team2_players).reshape(1, -1)
    probabilities = model.predict_proba(input_data)
    odds_team1_wins = probabilities[0][0]  # Probability of team 1 winning
    return odds_team1_wins

In [ ]:
d1 = ["Knight", "Magic Archer", "Goblin Drill", "The Log", "Tornado", "Ice Spirit", "Skeletons", "Cannon"]
d2 = ["Musketeer", "Skeletons", "The Log", "Fireball", "Ice Spirit", "Cannon", "Hog Rirder", "Ice Golem"]

#d1 much better
d1 = ["Goblin Barrel", "Royal Recruits", "Goblinstein", "Cannon Cart", "Goblin Gang", "Dart Goblin", "Arrows", "Cannon"]
d2 = ["Elixir Collector", "Barbarian Hut", "Lightning", "Musketeer", "Zap", "Heal Spirit", "Goblin Machine", "Knight"]
d1 = deck_to_nums(d1)
d2 = deck_to_nums(d2)
predict_team_odds(d1, d2)

In [ ]:
model.save_model('xgboost_model.json')

#LIGHT GBM GRADIENT BOOSTING

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
data = pd.read_csv("matches.csv")

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [ ]:
params = {
    "objective": "binary",
    "boosting_type": "gbdt",
    "metric": "auc",
    "device": "cpu",
    "learning_rate": 0.1,
    "num_leaves": 31,
    "max_depth": -1,
    "verbose": -1,
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, test_data],
    valid_names=["train", "val"],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(50)]
)

y_pred = model.predict(X_test)

y_pred_binary = (y_pred >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred_binary) * 100

print(f"Validation AUC: {auc:.4f}")
print(f"Validation Accuracy: {accuracy:.2f}%")

In [ ]:
model.save_model("lightgbm_model.txt")
print("Model saved as 'lightgbm_model.txt'")

In [ ]:
def predict_odds(team1, team2):
    features = team1 + team2
    features = pd.DataFrame([features])
    probability = model.predict(features)[0]
    return 1.0 - probability

In [ ]:
d1 = ["Knight", "Magic Archer", "Goblin Drill", "The Log", "Tornado", "Ice Spirit", "Skeletons", "Cannon"]
d2 = ["Musketeer", "Skeletons", "The Log", "Fireball", "Ice Spirit", "Cannon", "Hog Rirder", "Ice Golem"]

#d1 much better
d1 = ["Goblin Barrel", "Royal Recruits", "Goblinstein", "Cannon Cart", "Goblin Gang", "Dart Goblin", "Arrows", "Cannon"]
d2 = ["Elixir Collector", "Barbarian Hut", "Lightning", "Musketeer", "Zap", "Heal Spirit", "Goblin Machine", "Knight"]
d1 = deck_to_nums(d1)
d2 = deck_to_nums(d2)
predict_odds(d1, d2)

#NEURAL NET IMPLEMENTATION

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
data = pd.read_csv("matches.csv")
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

In [ ]:
class PredictorNN(nn.Module):
    def __init__(self, input_size):
        super(PredictorNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

input_size = X_train.shape[1]
model = PredictorNN(input_size).to(device)

criterion = nn.BCELoss()  # Binary Cross Entropy Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 20
batch_size = 1024
train_size = X_train.size(0)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for i in range(0, train_size, batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss/train_size:.6f}")

model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred >= 0.5).float()
    accuracy = (y_pred == y_test).sum().item() / y_test.size(0)
    print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
torch.save(model.state_dict(), "predictor_model.pth")
print("Model saved to predictor_model.pth")

In [ ]:
def predict_odds(team1, team2):
    input_data = np.array(team1 + team2).reshape(1, -1)
    input_data = scaler.transform(input_data)
    input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)
    with torch.no_grad():
        probability = model(input_tensor).item()
    return 1.0 - probability

In [ ]:
d1 = ["Knight", "Magic Archer", "Goblin Drill", "The Log", "Tornado", "Ice Spirit", "Skeletons", "Cannon"]
d2 = ["Musketeer", "Skeletons", "The Log", "Fireball", "Ice Spirit", "Cannon", "Hog Rirder", "Ice Golem"]

#d1 much better
d1 = ["Goblin Barrel", "Royal Recruits", "Goblinstein", "Cannon Cart", "Goblin Gang", "Dart Goblin", "Arrows", "Cannon"]
d2 = ["Elixir Collector", "Barbarian Hut", "Lightning", "Musketeer", "Zap", "Heal Spirit", "Goblin Machine", "Knight"]
d1 = deck_to_nums(d1)
d2 = deck_to_nums(d2)
predict_odds(d1, d2)